<a href="https://colab.research.google.com/github/AnandaSheva/expert_system/blob/main/plant_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.3 MB/s eta 0:00:00


In [2]:
#import library
import re
import json
import pdfplumber

In [3]:
def baca_data_tanaman(path_pdf):
    pdf = pdfplumber.open(path_pdf)
    page_tanaman = pdf.pages[0].extract_text()
    page_tanaman = page_tanaman.replace("\n", " ")

    pola = r"Nama Tanaman: ([\w\s/]+), Ketinggian Tempat \(mdpl\): (\d+) - (\d+), Curah Hujan \(mm/bulan\): (\d+) - (\d+), Bulan Tanam: ([\w\s,]+);"
    tanaman = re.findall(pola, page_tanaman)

    isi_tanaman = []
    for tanaman_data in tanaman:
        tanaman_info = {
            "nama_tanaman": tanaman_data[0].strip(),
            "ketinggian_tempat": (int(tanaman_data[1]), int(tanaman_data[2])),
            "curah_hujan": (int(tanaman_data[3]), int(tanaman_data[4])),
            "bulan_tanam": [bulan.strip() for bulan in tanaman_data[5].split(",")]
        }
        isi_tanaman.append(tanaman_info)

    return isi_tanaman

def cek_kesesuaian_tanaman(tanaman, ketinggian, curah_hujan, bulan):
    if not (tanaman["ketinggian_tempat"][0] <= ketinggian <= tanaman["ketinggian_tempat"][1]):
        return False

    if not (tanaman["curah_hujan"][0] <= curah_hujan <= tanaman["curah_hujan"][1]):
        return False

    if bulan.capitalize() not in tanaman["bulan_tanam"]:
        return False

    return True

def dapatkan_rekomendasi(data_tanaman, ketinggian, curah_hujan, bulan):
    rekomendasi = []
    tidak_sesuai = []

    for tanaman in data_tanaman:
        sesuai = cek_kesesuaian_tanaman(tanaman, ketinggian, curah_hujan, bulan)
        if sesuai:
            rekomendasi.append({
                "nama_tanaman": tanaman["nama_tanaman"]
            })
        else:
            tidak_sesuai.append({
                "nama_tanaman": tanaman["nama_tanaman"]
            })

    return rekomendasi, tidak_sesuai

def export_json(data, path):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [ ]:
def main():
    try:
        data_tanaman = baca_data_tanaman('/content/Tugas 2.pdf')

        print("\n=== Sistem Rekomendasi Tanaman ===")
        ketinggian = int(input("Masukkan ketinggian tempat (mdpl): "))
        curah_hujan = int(input("Masukkan curah hujan (mm/bulan): "))
        bulan = input("Masukkan bulan tanam: ")

        rekomendasi, tidak_sesuai = dapatkan_rekomendasi(
            data_tanaman, ketinggian, curah_hujan, bulan
        )

        hasil_rekomendasi = {
            "input_data": {
                "ketinggian": ketinggian,
                "curah_hujan": curah_hujan,
                "bulan_tanam": bulan
            },
            "hasil_rekomendasi": {
                "tanaman_sesuai": rekomendasi,
                "tanaman_tidak_sesuai": tidak_sesuai
            }
        }

        export_json(hasil_rekomendasi, '/content/data_tanaman.json')

        print("\nRekomendasikan tanaman:")
        if rekomendasi:
            for idx, tanaman in enumerate(rekomendasi, 1):
                print(f"{idx}. {tanaman['nama_tanaman']}")
        else:
            print("Tidak ada tanaman yang sesuai")

    except FileNotFoundError:
        print("Error: File PDF tidak ditemukan")
    except ValueError:
        print("Error: Input tidak valid. Pastikan input berupa angka untuk ketinggian dan curah hujan")
    except Exception as e:
        print(f"Error: Terjadi kesalahan - {str(e)}")

if __name__ == "__main__":
    main()